In [2]:
import os
import random
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN

def detect_and_extract_face_mtcnn(img_path, output_face_path=None, show_result=False, device='cpu'):
    """
    Detects a face in the given image using MTCNN and extracts it.
    Optionally saves the cropped face to 'output_face_path'.
    If show_result=True, displays the result via matplotlib.
    Returns the cropped face as a NumPy array (BGR) or None if no face is found.
    """
    # Initialize MTCNN (you can do this once outside the function if preferred)
    mtcnn = MTCNN(keep_all=True, device=device)

    # Read image with OpenCV (BGR format)
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        print(f"[ERROR] Could not load image {img_path}")
        return None

    # Convert BGR to RGB for MTCNN
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # Detect faces
    boxes, probs = mtcnn.detect(img_rgb)

    if boxes is None or len(boxes) == 0:
        print(f"[INFO] No face detected in {img_path}")
        return None

    # Find the face with the highest probability
    max_prob_idx = np.argmax(probs)
    box = boxes[max_prob_idx]  # [x1, y1, x2, y2]
    conf = probs[max_prob_idx]

    print(f"[INFO] Detected face in {img_path} with confidence: {conf:.3f}")

    # Convert bounding box to integers
    x1, y1, x2, y2 = [int(coord) for coord in box]
    
    # Clamp coordinates to image boundaries
    h, w, _ = img_bgr.shape
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)

    # Crop the face region
    face_cropped_bgr = img_bgr[y1:y2, x1:x2]

    # Optionally save the cropped face
    if output_face_path:
        cv2.imwrite(output_face_path, face_cropped_bgr)

    # Display the detection if requested
    if show_result:
        # Draw a rectangle on the original image (for visualization)
        cv2.rectangle(img_bgr, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Convert images for matplotlib
        img_bgr_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        face_cropped_rgb = cv2.cvtColor(face_cropped_bgr, cv2.COLOR_BGR2RGB)

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
        
        ax1.imshow(img_bgr_rgb)
        ax1.set_title("ID Image with Detected Face (MTCNN)")
        ax1.axis("off")

        ax2.imshow(face_cropped_rgb)
        ax2.set_title("Cropped Face")
        ax2.axis("off")

        plt.tight_layout()
        plt.show()

    return face_cropped_bgr


In [3]:

import cv2
from deepface import DeepFace

def verify_faces(face1_bgr, face2_bgr, model_name='Facenet', distance_metric='cosine'):
    """
    Verifies whether face1 and face2 belong to the same person using DeepFace.
    Returns a dict with keys: { 'verified', 'distance', 'threshold', ... } 
    or None if something went wrong.
    """
    if face1_bgr is None or face2_bgr is None:
        print("[ERROR] One of the input faces is None.")
        return None

    # DeepFace expects RGB arrays (or file paths)
    face1_rgb = cv2.cvtColor(face1_bgr, cv2.COLOR_BGR2RGB)
    face2_rgb = cv2.cvtColor(face2_bgr, cv2.COLOR_BGR2RGB)

    try:
        result = DeepFace.verify(
            img1_path=face1_rgb,  
            img2_path=face2_rgb,
            model_name=model_name,
            distance_metric=distance_metric,
            enforce_detection=False  # because we already have cropped faces
        )
        return result
    except Exception as e:
        print(f"[ERROR] DeepFace verification failed: {e}")
        return None
    
def verify_face_from_id(id_path, selfie_path):
    # Step 1: Detect
    face_id = detect_and_extract_face_mtcnn(img_path=id_path, output_face_path=None, show_result=False, device='cpu')
    face_selfie = detect_and_extract_face_mtcnn(img_path=selfie_path, output_face_path=None, show_result=False, device='cpu')

    # Step 2: Verify
    if face_id is not None and face_selfie is not None:
        verification_result = verify_faces(face_id, face_selfie, model_name='Facenet', distance_metric='cosine')
        
        if verification_result is not None:
            if verification_result['verified']:
                return True
            else:
                return False
        else:
            return None
    else:
        return None


2025-01-14 15:01:31.079016: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736859691.110955  162597 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736859691.120669  162597 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 15:01:31.151051: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:

def select_random_images(people_dir):
    # Select a random person from the directory
    person_names = [person_name for person_name in os.listdir(people_dir) if os.path.isdir(os.path.join(people_dir, person_name))]
    
    # Pick one random person
    selected_person = random.choice(person_names)
    
    # Get all images in the selected person's folder
    person_dir = os.path.join(people_dir, selected_person)
    person_images = [f for f in os.listdir(person_dir) if f.endswith('.jpg') or f.endswith('.png')]
    
    # Select two random images from the same person's folder
    selected_images = random.sample(person_images, 2)
    
    # Select one image from a different person
    different_person = random.choice([name for name in person_names if name != selected_person])
    different_person_dir = os.path.join(people_dir, different_person)
    different_person_images = [f for f in os.listdir(different_person_dir) if f.endswith('.jpg') or f.endswith('.png')]
    different_person_image = random.choice(different_person_images)
    
    # Return the selected images
    return (os.path.join(person_dir, selected_images[0]), os.path.join(person_dir, selected_images[1])), \
           os.path.join(person_dir, selected_images[0]), os.path.join(different_person_dir, different_person_image)

def test_face_verification(people_dir):
    # Initialize counters for KPIs
    tn = tp = fn = fp = 0

    # List to store results for each person
    results = []

    # Loop through each person directory in the 'Original Images' folder
    for person_name in os.listdir(people_dir):
        person_dir = os.path.join(people_dir, person_name)

        # Ensure it's a directory
        if os.path.isdir(person_dir):
            # Select images for testing
            same_person_images, test_image_same1, test_image_diff = select_random_images(people_dir)

            # Test if same person images match (TP)
            result_same = verify_face_from_id(same_person_images[0], same_person_images[1])
            if result_same is True:
                tp += 1  # True Positive
            else:
                fn += 1  # False Negative

            # Test if images from different people match (TN)
            result_diff = verify_face_from_id(test_image_same1, test_image_diff)
            if result_diff is True:
                fp += 1  # False Positive
            else:
                tn += 1  # True Negative

            # Store results for this person
            results.append({
                "Person": person_name,
                "TP": tp,
                "FP": fp,
                "TN": tn,
                "FN": fn
            })

    # Calculate additional metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Print and store metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1_score:.4f}")
    
    # Store results in a DataFrame and save to CSV
    df = pd.DataFrame(results)
    df.to_csv('face_verification_results.csv', index=False)
    return df, accuracy, precision, recall, f1_score


In [5]:

# Example usage
people_dir = '/mnt/Files/Facultate/Master/an2/sem1/CV/Proiect/KYC-application/test/Original Images/Original Images'  
df, accuracy, precision, recall, f1_score = test_face_verification(people_dir)


[INFO] Detected face in /mnt/Files/Facultate/Master/an2/sem1/CV/Proiect/KYC-application/test/Original Images/Original Images/Tom Cruise/Tom Cruise_19.jpg with confidence: 1.000
[INFO] Detected face in /mnt/Files/Facultate/Master/an2/sem1/CV/Proiect/KYC-application/test/Original Images/Original Images/Tom Cruise/Tom Cruise_34.jpg with confidence: 1.000


2025-01-14 15:01:38.683341: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


[INFO] Detected face in /mnt/Files/Facultate/Master/an2/sem1/CV/Proiect/KYC-application/test/Original Images/Original Images/Tom Cruise/Tom Cruise_19.jpg with confidence: 1.000
[INFO] Detected face in /mnt/Files/Facultate/Master/an2/sem1/CV/Proiect/KYC-application/test/Original Images/Original Images/Ellen Degeneres/Ellen Degeneres_46.jpg with confidence: 1.000
[INFO] Detected face in /mnt/Files/Facultate/Master/an2/sem1/CV/Proiect/KYC-application/test/Original Images/Original Images/Ellen Degeneres/Ellen Degeneres_48.jpg with confidence: 1.000
[INFO] Detected face in /mnt/Files/Facultate/Master/an2/sem1/CV/Proiect/KYC-application/test/Original Images/Original Images/Ellen Degeneres/Ellen Degeneres_19.jpg with confidence: 1.000
[INFO] Detected face in /mnt/Files/Facultate/Master/an2/sem1/CV/Proiect/KYC-application/test/Original Images/Original Images/Ellen Degeneres/Ellen Degeneres_48.jpg with confidence: 1.000
[INFO] Detected face in /mnt/Files/Facultate/Master/an2/sem1/CV/Proiect/KYC